In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e2/sample_submission.csv
/kaggle/input/playground-series-s3e2/train.csv
/kaggle/input/playground-series-s3e2/test.csv


importing train data


In [2]:
train_data=pd.read_csv('/kaggle/input/playground-series-s3e2/train.csv')
train_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


importing test data

In [3]:
test_data=pd.read_csv('/kaggle/input/playground-series-s3e2/test.csv')
test_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,15304,Female,57.0,0,0,Yes,Private,Rural,82.54,33.4,Unknown
1,15305,Male,70.0,1,0,Yes,Private,Urban,72.06,28.5,Unknown
2,15306,Female,5.0,0,0,No,children,Urban,103.72,19.5,Unknown
3,15307,Female,56.0,0,0,Yes,Govt_job,Urban,69.24,41.4,smokes
4,15308,Male,32.0,0,0,Yes,Private,Rural,111.15,30.1,smokes


importing libraries needed for One Hot Encoding the categorical columns

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

using all the categorical features of the dataset and performing one hot encoding

In [5]:
cat_features = [col for col in train_data.select_dtypes(include=['object', 'category'])]
cat_features.extend(['hypertension','heart_disease'])
X_cat_train=train_data[cat_features]
X_cat_test=test_data[cat_features]

In [6]:
print(cat_features)

['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'hypertension', 'heart_disease']


viewing the categorical features of the dataset

In [7]:
X_cat_train.head()

,gender,ever_married,work_type,Residence_type,smoking_status,hypertension,heart_disease
0,Male,Yes,Private,Urban,never smoked,0,0
1,Male,Yes,Private,Rural,formerly smoked,0,0
2,Female,Yes,Private,Rural,Unknown,0,0
3,Male,Yes,Private,Urban,never smoked,0,0
4,Female,No,Private,Rural,never smoked,0,0


printing the categorical features of the test dataset

In [8]:
X_cat_test.head()

,gender,ever_married,work_type,Residence_type,smoking_status,hypertension,heart_disease
0,Female,Yes,Private,Rural,Unknown,0,0
1,Male,Yes,Private,Urban,Unknown,1,0
2,Female,No,children,Urban,Unknown,0,0
3,Female,Yes,Govt_job,Urban,smokes,0,0
4,Male,Yes,Private,Rural,smokes,0,0


look for the float values

In [9]:
test_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,15304,Female,57.0,0,0,Yes,Private,Rural,82.54,33.4,Unknown
1,15305,Male,70.0,1,0,Yes,Private,Urban,72.06,28.5,Unknown
2,15306,Female,5.0,0,0,No,children,Urban,103.72,19.5,Unknown
3,15307,Female,56.0,0,0,Yes,Govt_job,Urban,69.24,41.4,smokes
4,15308,Male,32.0,0,0,Yes,Private,Rural,111.15,30.1,smokes


In [10]:
num=['age','avg_glucose_level','bmi']
X_num_train=train_data[num]
X_num_test=test_data[num]
X_num_train.head()

,age,avg_glucose_level,bmi
0,28.0,79.53,31.1
1,33.0,78.44,23.9
2,42.0,103.00,40.3
3,56.0,64.87,28.8
4,24.0,73.36,28.8


scaling and converting back to dataframe again

In [11]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
c1=X_num_train.columns
c2=X_num_test.columns
X_num_train=scaler.fit_transform(X_num_train)
X_num_test=scaler.transform(X_num_test)
X_num_train=pd.DataFrame(X_num_train,columns=c1)
X_num_test=pd.DataFrame(X_num_test,columns=c2)
X_num_train.head()

,age,avg_glucose_level,bmi
0,-0.625710,-0.373297,0.444397
1,-0.392544,-0.416084,-0.626698
2,0.027154,0.547988,1.813018
3,0.680018,-0.948757,0.102242
4,-0.812243,-0.615493,0.102242


In [12]:
X_num_train.dtypes

age                  float64
avg_glucose_level    float64
bmi                  float64
dtype: object

In [13]:
y_train=train_data['stroke']

In [14]:
X_cat1e_train=pd.get_dummies(X_cat_train)
X_cat1e_train.head()

,hypertension,heart_disease,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
2,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0
3,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
4,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0


In [15]:
X_cat1e_test=pd.get_dummies(X_cat_test)
X_cat1e_test.head()

,hypertension,heart_disease,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0
1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0
2,0,0,1,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0
3,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1
4,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1


In [16]:
X_train=pd.concat([X_cat1e_train,X_num_train],axis=1)
X_test=pd.concat([X_cat1e_test,X_num_test],axis=1)
X_train.head()

,hypertension,heart_disease,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,...,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,age,avg_glucose_level,bmi
0,0,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,-0.625710,-0.373297,0.444397
1,0,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,-0.392544,-0.416084,-0.626698
2,0,0,1,0,0,0,1,0,0,1,...,0,1,0,1,0,0,0,0.027154,0.547988,1.813018
3,0,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0.680018,-0.948757,0.102242
4,0,0,1,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,-0.812243,-0.615493,0.102242


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

lasso = Lasso(random_state=0, max_iter=10000)
alphas = np.logspace(-4, -0.5, 30)
#alphas = np.array([0, 0.01, 0.02, 0.03, 0.05, 0.1, 0.5, 1.0, 3.0, 5.0])
grid1 = GridSearchCV(estimator=lasso, cv=5, scoring='roc_auc', param_grid=dict(alpha=alphas))
grid1.fit(X_train, y_train)
print('Best value for alpha is: ', grid1.best_estimator_.alpha)

Best value for alpha is:  0.0196382800192977


Scaling also done so need of normalizing

In [18]:
lassoReg = Lasso(alpha=0.0196,normalize=False)
lassoReg.fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,


Lasso(alpha=0.0196, normalize=False)

printing coefficients for formality

In [19]:
# Checking the coefficients
print(lassoReg.coef_)

[ 0.          0.         -0.          0.          0.          0.
 -0.         -0.         -0.         -0.          0.          0.
  0.         -0.          0.          0.         -0.          0.
  0.03167165  0.00665272 -0.        ]


In [20]:
y_pred = lassoReg.predict(X_test)

time to submit predictions

In [21]:
output=pd.DataFrame({'id':test_data.id,'stroke':y_pred})
output.to_csv('submissions.csv',index=False)
print("Successful submission")

Successful submission


time to check submission table